<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub (https://github.com/Caltech-IPAC/Montage).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mDiffFitExec

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

This page is focused on the details of one of these modules: mDiffFitExec.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.

<b>Note:</b> The MontagePy python package has no external dependencies. We include other utilities on this page to aid in visualizing MontagePy package results.

In [1]:
from MontagePy.main import mDiffFitExec, mViewer

help(mDiffFitExec)

Help on built-in function mDiffFitExec in module MontagePy.main:

mDiffFitExec(...)
    mDiffFitExec loops over a list of image overlaps, computing then fitting the differences.
    
    Parameters
    ----------
    path : str
        Path to raw image directory.
    tblfile : str
        Table file list of input images.
    template : str
        FITS header file used to define the desired output.
    diffdir : str
        Directory for temporary difference files.
    fitfile : str
        Table file for output difference fit info.
    keepAll : bool, optional
        Flag to keep temporary difference images.
    levelOnly : bool, optional
        Flag to fit level of differences only, not slopes.
    noAreas : bool, optional
        Flag indicating there are no area images.
    debug : int, optional
        Debugging output flag.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    count : i

## mDiffFitExec Example

mDiffFitExec is a general utility for generating and fitting a set of image differences. It's most common use is as part of the process of evaluating overlap area in a set of images as part of background rectification. That process consists of taking the output of the overlaps calculation (mOverlaps) and for each overlap pair doing the difference (mDiff), fitting it to determine offset between those two images (mFitplane).

There are two ways to do this in bulk. We can either generate all the differences first, then fit them all or we can loop over the differences, generating and fitting that difference before moving on to the next. This first approach uses more disks space as all the differences have to be stored in between steps. The second can clean up as it goes.

Obviously the second approach is preferable unless you want to keep the differences (possibly for quality evaluation).

mDiffFitExec is an instantiation of the second approach. It runs mDiff on every pair in the input list and then fits it with a plane.

At this point in the processing, we already have a set of "like" images (same projection), and have made a list of all the overlaps (normally using mOverlaps).  mDiffFitExec loops over these differences, generating each difference (in a working directory) and fitting a plane to it.

mDiffFitExec cleans up the working files as it goes.

In [4]:
import os

try:
   os.makedirs('work/M17')
except:
   pass

rtn = mDiffFitExec('M17/projected', 
                   'M17/diffs.tbl', 
                   'M17/M17.hdr',
                   'M17/diffs',
                   'work/M17/fits.tbl')
print(rtn)

{'status': '0', 'count': 128, 'diff_failed': 0, 'fit_failed': 0, 'warning': 0}


The output is a set of planes representing the difference between two original images.  This set will be sent to mBgModel for modelling into a set of corrections to the original images to bring the backgrounds in line.

In [6]:
import numpy as np
import pandas as pd
from astropy.io import ascii

ipactable = ascii.read('work/M17/fits.tbl').to_pandas()

ipactable

,plus,minus,a,b,c,crpix1,crpix2,xmin,xmax,ymin,ymax,xcenter,ycenter,npixel,rms,boxx,boxy,boxwidth,boxheight,boxang
0,0,8,1.322410e-03,4.731330e-04,0.333613,-1711.0,1756.5,1711,1789,-1755,-935,1749.95,-1343.56,52522,0.670273,1750.0,-1346.0,821.2,77.6,-90.1
1,0,9,-2.744710e-03,1.657260e-03,7.368920,-1713.0,988.5,1713,1789,-988,-733,1751.34,-861.76,15999,0.654584,1751.5,-860.5,256.0,77.0,90.0
2,0,13,2.935100e-03,3.910380e-03,-0.381983,-1286.0,986.5,1286,1361,-985,-732,1323.70,-861.28,16024,0.668661,1324.0,-859.0,255.0,76.0,90.0
3,0,14,-2.499470e-03,8.967710e-04,4.451240,-1283.0,1755.5,1285,1361,-1754,-933,1322.63,-1341.89,52364,0.654649,1322.4,-1344.5,822.2,77.5,89.9
4,0,21,1.578360e-07,2.346880e-06,-1.128660,-1287.0,786.5,1287,1789,-785,-732,1545.35,-760.28,19144,0.000454,1538.0,-759.2,502.1,54.4,-0.1
5,0,24,1.053080e-07,-1.312730e-06,0.559221,-1284.0,1756.5,1285,1787,-1755,-1702,1536.96,-1728.68,19964,0.000395,1535.5,-1729.3,503.1,54.4,-0.1
6,0,29,-1.314510e-03,7.775720e-04,-78.688200,-1710.0,994.5,1711,1789,-993,-733,1749.74,-865.03,16535,0.669547,1749.8,-863.6,262.2,79.8,-90.3
7,0,31,2.390970e-03,1.081350e-03,-82.292900,-1708.0,1756.5,1709,1789,-1755,-941,1748.59,-1347.27,53068,0.709566,1748.6,-1349.0,815.2,80.2,-90.1
8,1,6,-1.218190e-04,-1.797420e-04,-3.924660,426.0,789.5,-425,-351,-789,32,-387.28,-346.58,45487,0.742056,-388.0,-378.0,823.0,76.0,90.0
9,1,11,-2.616330e-03,-4.350420e-04,-0.515992,-2.0,789.5,2,77,-789,31,39.92,-384.16,50323,0.751111,40.0,-378.5,822.0,76.0,90.0


&nbsp;<p/>

## Error Handling

If mDiffFitExec encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a table that doesn't exist: 


In [7]:
rtn = mDiffFitExec('M17/projected', 
                   'M17/unknown.tbl', 
                   'M17/M17.hdr',
                   'M17/diffs',
                   'work/M17/fits.tbl')
print(rtn)

{'status': '1', 'msg': b'Invalid diffs metadata file: M17/unknown.tbl'}


&nbsp;

## Classic Montage: mDiffFitExec as a Stand-Alone Program


### mDiffFitExec Unix/Windows Command-line Arguments

<p>mDiffFitExec can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mDiffFitExec [-d] [-l(evel-only)] [-n(o-areas)] [-p projdir] [-s statusfile] diffs.tbl template.hdr diffdir fits.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mDiffFitExec can be accessed as a library function:</p>


<pre>
/*-*****************************************************************/
/*                                                                 */
/*  mDiffFitExec                                                   */
/*                                                                 */
/*  This routine combines the mDiff and mFit functionality and     */
/*  optionally discards the difference images as it goes (to       */
/*  minimize the file space needed).   It uses the table of        */
/*  oerlaps found by mOverlaps, running mDiff, then mFitplane      */
/*  on the difference images.  These fits are written to an        */
/*  output file which is then used by mBgModel.                    */
/*                                                                 */
/*   char *path        Path to images to be diffed.                */
/*   char *tblfile     Table file list of images to diff.          */
/*   char *template    FITS header file used to define the desired */
/*                     output.                                     */
/*                                                                 */
/*   char *diffdir     Directory for temporary output diff files.  */
/*   char *fitfile     Table file for output difference fits info. */
/*   int keepAll       Flag to keep temporary diff images.         */
/*   int levelOnly     Flag to fit level of diff only, not slopes. */
/*   int noAreas       Flag indicating there are no area images.   */             
/*   int debug         Debug flag.                                 */
/*                                                                 */
/*******************************************************************/

struct mDiffFitExecReturn *mDiffFitExec(char *path, char *tblfile, char *template, char *diffdir,
                                        char *fitfile, int keepAll, int levelOnly, int noAreas, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mDiffFitExecReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   int    count;         // Number of differences                
   int    diff_failed;   // Number of differences that failed
   int    fit_failed;    // Number of fits to differences that failed
   int    warning;       // Number of fits to differences that produced warnings
};
</pre>